In [ ]:
# Imports not from the google drive
# stdlib
import copy
import math
import os
import random
import sys

# external
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import tensorflow as tf
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

!pip install import_ipynb
# external
import import_ipynb

# Imports from the google drive
from google.colab import drive

drive.mount("/content/drive")

%cd /content/drive/Shareddrives/Space Systems Division/Teams/FINCH/Payload System/Science/Projects/SC-4: Data Processing/Destriping/Noise Functions
file_dir = "/content/drive/Shared drives/Space Systems Divison/FINCH Mission/Payload System/Science/Projects/SC-4: Data Processing/Destriping/Noise Functions"
sys.path.append(os.path.abspath(file_dir))
# external
import apply_stripes

%cd /content/drive/Shareddrives/Space Systems Division/Teams/FINCH/Payload System/Science/Projects/SC-4: Data Processing/Destriping/Destriping Models
file_dir = "/content/drive/Shared drives/Space Systems Divison/FINCH Mission/Payload System/Science/Projects/SC-4: Data Processing/Destriping/Destriping Models"
sys.path.append(os.path.abspath(file_dir))
# external
import noise_estimation

%cd /content/drive/Shareddrives/Space Systems Division/Teams/FINCH/Payload System/Science/Projects/SC-4: Data Processing/Destriping/Destriping Models/SURE CNN/Colab Notebooks/HSI_Denoising_SURE_CNN-master
file_dir = "/content/drive/Shared drives/Space Systems Divison/FINCH Mission/Payload System/Science/Projects/SC-4: Data Processing/Destriping/Destriping Models/SURE CNN/Colab Notebooks/HSI_Denoising_SURE_CNN-master"
sys.path.append(os.path.abspath(file_dir))
# external
from models.skipnet import *
from utils.common import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 3.2 MB/s 
Mounted at /content/drive
/content/drive/Shareddrives/Space Systems Division/Teams/FINCH/Payload System/Science/Projects/SC-4: Data Processing/Destriping/Noise Functions
importing Jupyter notebook from apply_stripes.ipynb
/content/drive/Shareddrives/Space Systems Division/Teams/FINCH/Payload System/Science/Projects/SC-4: Data Processing/Destriping/Destriping Models
importing Jupyter notebook from noise_estimation.ipynb
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/Shareddrives/Space Systems Division/Teams/FINCH/Payload System/Science/Projects/SC-4: Data Processing/Destriping/Destriping Models/SURE CNN/Colab Notebooks/HSI_Denoising_SURE_CNN-master


In [ ]:
def import_indian_pines_data(to_import=True):
    """
    import Indian Pines dataset from Science Google Drive
    Args:
      import: if True import the dataset from Googel Drive, if false doesn't import
      To change the location of import to github

    Return: print statement of whether the dataset is imported properly or not
    """
    if to_import == True:
        # data get data to add stripe
        #!git clone https://github.com/spacesys-finch/Science #if says error uncomment this line
        # error cloning, so uploaded the data to google drive instead
        data = np.load(
            "/content/drive/Shareddrives/Space Systems Division/Teams/FINCH/Payload System/Science/Projects/SC-4: Data Processing/Destriping/Datasets/indian_pine_array.npy"
        )

        # changing from callibrated value to radiance
        # radiance_data_np = (data-1000)/500
        radiance_data_np = data.astype(np.float32)
        return radiance_data_np

In [ ]:
# write a function for nomalizing the values in the numpy array


def normalize(numpy_arr_one, numpy_arr_two):
    """
    Normalize the values in numpy_arr_one and numpy_arr_two into values between -1 and 1
    Args:
      numpy_arr_one, numpy_arr_two: numpy arrays of the data to normalize
    Return: numpy_arr_one_norm, numpy_arr_two_norm: two arrays that are normalized
    """

    max_num = max(np.max(numpy_arr_one), np.max(numpy_arr_two))
    divisor = np.ones(numpy_arr_one.shape) * max_num
    numpy_arr_one_norm = np.divide(numpy_arr_one, divisor)
    numpy_arr_two_norm = np.divide(numpy_arr_two, divisor)
    return numpy_arr_one_norm, numpy_arr_two_norm, max_num

In [ ]:
# reshape data to (1, data.shape[0], data.shape[1], data.shape[2])
def reshape(data):
    """
    Input: 3D Numpy array, Output: Reshaped 4D Numpy array
    Reshape HSI data from 3D to 4D to pass into CNN
    Ex. original data shape (20, 30, 40) will turn into (1, 20, 30, 40)
    """
    data_reshaped = data.reshape(1, data.shape[0], data.shape[1], data.shape[2])
    return data_reshaped

In [ ]:
def define_model(striped_data):
    """
    Define the models and the necessary parameters of the model

    Args: None
    Return:
      mymodel: model of the defined properties
      lr: alpha value
      myoptimizer: optimizer of choice
      loss_object: type of loss object
    """
    # Define model
    mymodel = skip(ndown=5, channel=striped_data.shape[-1])
    lr = 0.001
    myoptimizer = tf.keras.optimizers.Adam(lr)
    loss_object = tf.keras.losses.MeanSquaredError()
    return mymodel, lr, myoptimizer, loss_object

In [ ]:
# Calculate gradient
@tf.function
def grad(model, inputs, targets):
    sure = True
    with tf.GradientTape() as tape:
        loss_value, div_value = losshyper(
            model, inputs, targets, sigma=sigmaest, sure=sure
        )
    return loss_value, div_value, tape.gradient(loss_value, model.trainable_variables)

In [ ]:
# function for calculating mean ssim (Structural Similarity Index)
def meanssim(X, Y):
    """
    Calculate the mean SSIM of the two images X and Y

    Args:
    x: first image to compare
    y: second image to compare with

    return
    np.mean(bandssim): mean similarity of the two images x and y
    """
    bandssim = []
    for i in range(X.shape[2]):
        bandssim.append(
            ssim(
                X[:, :, i],
                Y[:, :, i],
                data_range=1.0,
                K1=0.01,
                K2=0.03,
                gaussian_weights=True,
                sigma=1.5,
                use_sample_covariance=False,
            )
        )
    return np.mean(bandssim)

In [ ]:
def train_model(train, model_file_path, striped_data, clean_data, num_epochs):
    """
    This function train the model and save the model

    Args:
      train: if true, train, if false not train
      model_file_path: filepath that the model is saved at
      striped_data: striped_data as numpy array with the shape (1, data.shape[0], data.shape[1], data.shape[2])
      clean_data: clean data as numpy array with the shape (1, data.shape[0], data.shape[1], data.shape[2])

    Return:
      train_loss_results: list of the training loss
      destriped_psnr: list of the psnr between the

    """
    if train == True:
        mymodel, lr, myoptimizer, loss_object = define_model(striped_data)
        sure = True
        # start training
        train_loss_results = []
        train_div_results = []
        destriped_psnr = []
        destriped_ssim = []
        curr_psnr = 0
        sure = True
        # for loop for going through each epoch
        for epoch in range(num_epochs):
            loss_label = "SURE"
            # get gradient for the current model
            loss_value, div_value, grads = grad(mymodel, striped_data, striped_data)
            # apply gradient to the optimizer
            myoptimizer.apply_gradients(zip(grads, mymodel.trainable_variables))
            img_out = mymodel.predict(striped_data)
            # if the pnsr between the clean image and the output from the model is higher than the previous output, save the current model
            if psnr(clean_data[0], img_out[0]) > curr_psnr:
                print("Saving Model")
                tf.keras.models.save_model(mymodel, model_file_path)
                curr_psnr = psnr(clean_data[0], img_out[0])
            curr_ssim = meanssim(clean_data[0], img_out[0])

            print("This is running epoch %d and Loss %f" % (epoch, loss_value))
            print("PSNR: " + str(curr_psnr))
            print("SSIM: " + str(curr_ssim))
            print(
                "Iteration %05d    Loss %f     PSNR %f    SSIM: "
                % (epoch, loss_value, curr_psnr),
                "\r",
                end="",
            )
            # updating loss, div, ssim
            train_loss_results.append(loss_value)
            train_div_results.append(div_value)
            destriped_psnr.append(curr_psnr)
            destriped_ssim.append(curr_ssim)

        return train_loss_results, destriped_psnr, destriped_ssim

    else:
        print("Does not train")
        return [0], [0]

In [ ]:
def load_model_and_predict(filepath, striped_data):
    """
    This function load the saved model from the filepath and apply the model to striped_data to get the output

    Args:
      filepath: filepath that the model is saved in, the model should be of type .hdf5
      striped_data: striped_data in the format of numpy array with the shape (1, data.shape[0], data.shape[1], data.shape[2])

    Return:
      bestmodel: the model that is loaded from the filepath
      out_best: the output from passing striped_data into the loaded model
    """
    # set the model to load the trained model
    bestmodel = tf.keras.models.load_model(filepath)
    # pass the noisy image into the model to get the output from the trained model
    out_best = bestmodel.predict(striped_data)
    return bestmodel, out_best

In [ ]:
# load data and add stripe/noise
ip_radiance_data_np = import_indian_pines_data()
ip_striped_data_np = ip_radiance_data_np
ip_radiance_data_np, ip_striped_data_np, factor = normalize(
    ip_radiance_data_np, ip_radiance_data_np
)
num_stripes = []

num_bands = 200  # number of bands in the HSI
num_of_stripes = 25  # number of stripes for each band

for i in range(num_bands):
    num_stripes.append(num_of_stripes)

# add stripes, change to other type of noise
ip_striped_data_np = apply_stripes.add_basic_stripes(ip_radiance_data_np, num_stripes)

# plot the striped images
band = 12
plt.subplot(121)
plt.imshow(np.clip(ip_radiance_data_np[:, :, band], 0, 1))
plt.subplot(122)
plt.imshow(np.clip(ip_striped_data_np[:, :, band], 0, 1))
plt.show()

# reshape radiance data and striped data
ip_radiance_data = reshape(ip_radiance_data_np)
ip_radiance_data = ip_radiance_data.astype(np.float32)
ip_striped_data = reshape(ip_striped_data_np)
ip_striped_data = ip_striped_data.astype(np.float32)

# calculate noise estimation
sigmaest = noise_estimation.noise_estimate(ip_radiance_data_np)
sigmaest = sigmaest.astype(np.float32)

In [ ]:
# change filename of the model saved
filepath = (
    "./results/best model/Demo/"
    + "sure"
    + "indian_pine_noise_estimate_function"
    + ".hdf5"
)

In [ ]:
to_train = True
sure = True
num_epochs = 50
train_loss_results, destriped_psnr, destriped_ssim = train_model(
    to_train, filepath, ip_striped_data, ip_radiance_data, num_epochs
)
PUbestmodel, out_best = load_model_and_predict(filepath, ip_striped_data)

In [ ]:
# show the output image
fig1, (ax, ax1, ax2, ax3) = plt.subplots(4, sharex=False, figsize=(6, 12))
ax.imshow(np.clip(out_best[0][:, :, 133], 0, 1))
ax.set_title("Denoised  image, max PSNR=" + str(np.round(np.max(destriped_psnr), 2)))
ax.axis("off")
# plot train loss and divergence
ax1.plot(range(len(train_loss_results)), train_loss_results, label="train loss")
ax1.set_title("Training loss using SURE")
ax1.legend()
ax1.grid()

# plot PSNR
ax2.plot(destriped_psnr, label="destriped PSNR")
ax2.set_xlabel("Iterations")
ax2.set_title("PSNR using SURE")
ax2.legend()
ax2.grid()

# plot PSNR
ax3.plot(destriped_ssim, label="destriped SSIM")
ax3.set_xlabel("Iterations")
ax3.set_title("PSNR using SURE")
ax3.legend()
ax3.grid()

# show the images (clean, noisy, output from the trained model)
band = 12
plt.subplot(131)
plt.imshow(np.clip(ip_radiance_data_np[:, :, band], 0, 1))
plt.subplot(132)
plt.imshow(np.clip(ip_striped_data_np[:, :, band], 0, 1))
plt.subplot(133)
plt.imshow(np.clip(out_best[0][:, :, 133], 0, 1))
plt.show()
print(str(ip_radiance_data_np.shape))
print("Original PSNR: " + str(psnr(ip_radiance_data[0], ip_striped_data[0])))
print("Destriped PSNR: " + str(psnr(ip_radiance_data[0], out_best[0])))